In [26]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.metrics import confusion_matrix

import utils.text_processing as tp

In [12]:
pmethod = 'typos'

In [13]:
%store -r important_words
%store -r sentence_packages
%store -r ds
%store -r advds
%store -r extended_original_predictions
%store -r extended_modified_predictions

In [14]:
len(extended_modified_predictions)

2092

In [15]:
def generate_results_df(pmethod, ds, advds, extended):  
    
    if       
    results = pd.DataFrame({
     'Perturbation Method': [pmethod],
     'Tokenizer': ['nlptown/bert-base-multilingual-uncased-sentiment'], 
     'Model' : ['nlptown/bert-base-multilingual-uncased-sentiment'], 
     'Dataset':['TripAdvisor Hotel Reviews'], 
     'Output lables': ['Range from 0 to 4 - 0 = NEG; 4 = POS'],
     'Items in original dataset': len(ds),
     'Number of modifyable items in original dataset': len(advds),
     'Items in adversarial dataset': len(extended),
     'Percentage': (len(advds)/len(ds)*100)})
    
    return results.T

SyntaxError: invalid syntax (<ipython-input-15-1a944b694641>, line 3)

In [16]:
results_typo = generate_results_df(pmethod, ds, advds, extended_modified_predictions)
results_typo

NameError: name 'generate_results_df' is not defined

### Dataloader

In [37]:
sentences, aspect_category_sentiments, (idx2aspectlabel, idx2sentilabel) = semeval_to_aspectsentiment_hr(filename)

In [1]:
len(sentences)

NameError: name 'sentences' is not defined

In [39]:
len(aspect_category_sentiments)

1974

In [40]:
len((idx2aspectlabel))

81

In [41]:
idx2aspectlabel

{0: 'BATTERY#GENERAL',
 1: 'BATTERY#MISCELLANEOUS',
 2: 'BATTERY#OPERATION_PERFORMANCE',
 3: 'BATTERY#QUALITY',
 4: 'COMPANY#GENERAL',
 5: 'CPU#MISCELLANEOUS',
 6: 'CPU#OPERATION_PERFORMANCE',
 7: 'CPU#QUALITY',
 8: 'DISPLAY#DESIGN_FEATURES',
 9: 'DISPLAY#GENERAL',
 10: 'DISPLAY#MISCELLANEOUS',
 11: 'DISPLAY#OPERATION_PERFORMANCE',
 12: 'DISPLAY#QUALITY',
 13: 'DISPLAY#USABILITY',
 14: 'FANS_COOLING#DESIGN_FEATURES',
 15: 'FANS_COOLING#OPERATION_PERFORMANCE',
 16: 'FANS_COOLING#QUALITY',
 17: 'GRAPHICS#DESIGN_FEATURES',
 18: 'GRAPHICS#GENERAL',
 19: 'GRAPHICS#MISCELLANEOUS',
 20: 'GRAPHICS#QUALITY',
 21: 'HARDWARE#QUALITY',
 22: 'HARD_DISC#DESIGN_FEATURES',
 23: 'HARD_DISC#QUALITY',
 24: 'KEYBOARD#DESIGN_FEATURES',
 25: 'KEYBOARD#GENERAL',
 26: 'KEYBOARD#OPERATION_PERFORMANCE',
 27: 'KEYBOARD#QUALITY',
 28: 'KEYBOARD#USABILITY',
 29: 'LAPTOP#CONNECTIVITY',
 30: 'LAPTOP#DESIGN_FEATURES',
 31: 'LAPTOP#GENERAL',
 32: 'LAPTOP#MISCELLANEOUS',
 33: 'LAPTOP#OPERATION_PERFORMANCE',
 34: 'LAPTO

In [46]:
important_words_nr1 = []
important_words_nr2 = []
important_words_nr3 = []

In [42]:
sentence_packages = [{'original_sentence': 'This computer is absolutely AMAZING!!!', 'modified_sentences': [('This', 'computer is absolutely AMAZING!!!'), ('computer', 'This is absolutely AMAZING!!!'), ('is', 'This computer absolutely AMAZING!!!'), ('absolutely', 'This computer is AMAZING!!!'), ('AMAZING!!!', 'This computer is absolutely')]}, {'original_sentence': '10 plus hours of battery...', 'modified_sentences': [('10', 'plus hours of battery...'), ('plus', '10 hours of battery...'), ('hours', '10 plus of battery...'), ('of', '10 plus hours battery...'), ('battery...', '10 plus hours of')]}]

In [43]:
len(sentence_packages)

2

In [67]:
original_results = [{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'}]

modified_results = [{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'},{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'CPU#BLALLAL': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'},{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRdfsaERAL': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'},{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'},{'LAPTOP#GENERAL': 'POS'},
 {'BATTERY#OPERATION_PERFORMANCE': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'CPU#OPERATION_PERFORMANCE': 'POS', 'GRAPHICS#GENERAL': 'POS'},
 {'HARD_DISC#hellooo': 'POS'}]

In [68]:
important_words_1 = []
important_words_2 = []
important_words_3 = []

for e, package in enumerate(sentence_packages): 
    important_words_nr1 = []
    important_words_nr2 = []
    important_words_nr3 = []
    original_sentence = package['original_sentence']
    # print('new package: ' + original_sentence)

    original_result = original_results[e]
    
    for key, value in original_result.items():
        original_entity = key.split("#")[0]
        original_attribute = key.split("#")[1]
        original_sentiment = value

    for item in package['modified_sentences']:
        word = item[0]
        # sentence = item[1]
        # modified_result = tp.predict_sentiment(model, tokenizer, sentence)
        modified_result = modified_results[e]       
        
        for key, value in modified_result.items():
            mod_entity = key.split("#")[0]
            mod_attribute = key.split("#")[1]
            mod_sentiment = value
        
        modification = 0
        if mod_entity != original_entity:
            modification += 1
        if mod_attribute != original_attribute:
            modification += 1
        if mod_sentiment != original_sentiment:
            modification += 1
        
        if modification > 2:
            important_words_nr1.append(word)
        elif modification == 2:
            important_words_nr2.append(word)
        else: 
            important_words_nr3.append(word)


            
            
    important_words_1.append(important_words_nr1)
    important_words_2.append(important_words_nr2)
    important_words_3.append(important_words_nr3)



In [69]:
print(important_words_1)

[[], []]


In [70]:
print(important_words_2)

[[], []]


In [71]:
print(important_words_3)

[['This', 'computer', 'is', 'absolutely', 'AMAZING!!!'], ['10', 'plus', 'hours', 'of', 'battery...']]
